# Notes for Readers / Markers

This notebook is a combination of individual note books. We also did not remove or clean the code for presentation purposes. As such, the flow may not be totally coherant, and there may be examples of very poor or even nonsensical code snippets used for testing or checking for robustness / sanity checks.

# Notes for us:

Paper source: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5009332/

Data source: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5009332/bin/srep32586-s2.xls

Paper for original analysis:

https://osf.io/wzhe7/

https://bmcplantbiol.biomedcentral.com/articles/10.1186/s12870-019-2039-9


Goals for us:

Compare normal regression to the quantile regression from the paper and critically analyse.

Secondly, obviously these metrics will have correlations, and salt may affect 1 and only indirectly the other. For example, there may be a plausible connect between flowering time and ear number per plant. These may both be directly infleunced by an overall salt-tolerance factor, as well as each other. 

Structured equation modelling (SEM) allows us to model and test these correlations and define a general "salt-tolerance" factor. 

Theoretically, we can then use this factor to compare plant strains to see if any are more or less salt tolerant. Sample sizes we strain are relatively small, which is common in plant sciences, which allows for good critical discussion about the limitations of statistical inference. 

From here, if this is done in time, we can then include bayesian modelling to see if the observed data sufficiently updates our priors (control data + assumptions) to the point of significance. 





# Intro

Why salt tolerance? Climate change, feeding people. Allows for testing plant genetics and development of salt resistance crops.

Features: 
- Flowering time (HEA)
- maturity time (MAT)
- ripening period (RIP)
- plant height (HEI)
- thousand grain mass (TGW)
- ear number per plant (EAR)
- grain number per ear (GPE)
- dry mass per m2 (DRY_WT)
- yield (YLD)
- harvest index (HI).

### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import statsmodels.api as sm

%config InlineBackend.figure_format = "retina"

from sklearn.decomposition import PCA, FactorAnalysis
from factor_analyzer import FactorAnalyzer
import semopy

### Read Data

In [ ]:
data = pd.read_excel("https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5009332/bin/srep32586-s2.xls",skiprows=1)
color = ["Yellow" if x == "Control" else "Blue" for x in data["Condition"]]

In [ ]:
data.head()

#### General scatter matrix

We see that on an overall level, correlations are relatively non-existant. However, once we break the data down into groups (Treatment, Year), the correlations become much more apparent.

We exploit this structure to define an overall "salt-tolerance" factor. Finally we compare the correlation of the Salt tolerance factor to the Salt tolerance index used in the publication.

In [ ]:
pd.plotting.scatter_matrix(data, alpha=0.2, figsize=(20,10), c=color);

In [ ]:
pd.plotting.scatter_matrix(data[data["Condition"]=="Control"], alpha=0.2, figsize=(20,10));

In [ ]:
pd.plotting.scatter_matrix(data[(data["Condition"]=="Control") & (data["Year"]==2014)], alpha=0.2, figsize=(20,10));

### Pre-processing.

We want to model the response to the salt treatment, and see how each variable responds to the change. To do this, we essentially ask "How much do you change with when we add salt". To be able to compare with the (working, and used) SWP metric, and to produce a variable that is on the same scale as the original salt tolerance factor, we divide the treatment group by the square root of the control group.

The dataframe below is the result:

In [ ]:
dt = data.groupby(["Line","Year","Condition"]).mean().reset_index()
dt_c = dt[dt["Condition"]== "Control"].loc[:,dt.columns != 'Condition'].groupby(["Line","Year"]).mean()
dt_t = dt[dt["Condition"]!= "Control"].loc[:,dt.columns != 'Condition'].groupby(["Line","Year"]).mean()

# Center
#dt_t = dt_t-dt_t.mean()
#dt_c = dt_c-dt_c.mean()

df = dt_t / np.sqrt(dt_c)
dt_c.columns = [s[0:3].strip() for s in list(df.columns)]
dt_t.columns = [s[0:3].strip() for s in list(df.columns)]
df.columns = [s[0:3].strip() for s in list(df.columns)]

ST_1 = dt_t["YLD"] / np.sqrt(dt_c["YLD"])

In [ ]:
df

In [ ]:
pd.plotting.scatter_matrix(df, alpha=0.2, figsize=(20,10));

In [ ]:
#prepare data in for statsmodel, as single with all data is needed
df['ST_1'] = ST_1  

df1 = df.reset_index()

#include year into dataset might be used
df1['YEAR'] = [1 if x == 2014 else 0 for x in df1["Year"]]

df_train = df1.sample(frac = 0.67, random_state=1) #train data
df_test = df1.drop(df_train.index) #test data (whole data - train data)

y_test = df_test['ST_1'] 
X_test = df_test.drop(['ST_1'],axis=1)

### Run Linear regression and interpret

We start off with a simple OLS regression to the mean. Regressing HEA + MAT + RIP + HEI  + EAR + GPE +HI +TGW +DRY + Year on ST_1.

In [ ]:
lin_reg = smf.ols(formula='ST_1 ~ HEA + MAT + RIP + HEI  + EAR + GPE +HI +TGW +DRY',data =df_train).fit() #specify our model  and fit 
y_pred = lin_reg.predict(X_test) #make predictions

mean_squared_error(y_test, y_pred) #calculate different performance meassures
mean_absolute_error(y_test, y_pred)
print(lin_reg.summary())

Unfortunatly our Regression suffers from multicollinearity. This is only a problem, if we want to interpret our coefficients, if the Regression for prediction this should be fine. Furthermore we should investigate if our model assumptions are meet.

In [ ]:
#calculate the residuals
residuals = y_test - y_pred

# plot the histogram
f = plt.figure()

plt.hist(residuals,color='#c9e5de')
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.show()
f.savefig('hist.png',bbox_inches='tight',dpi=500)

Unfortunatly our Regression also suffers from heteroscedicity, rendering our predictions also useless. This will be further investigated in the R-code, with more beatiful plots.

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel('Observed Values')
plt.ylabel('Predicted Values')
plt.show()

We deal with Mulitcollinarirty by dropping several columns.

In [ ]:
lin_reg = smf.ols(formula='ST_1 ~ HEA + MAT + RIP + HEI  + EAR + GPE +HI',data =df_train).fit() #specify our model  and fit 
y_pred = lin_reg.predict(X_test) #make predictions
mean_squared_error(y_test, y_pred) #calculate different performance meassures
mean_absolute_error(y_test, y_pred)
print(lin_reg.summary())

## Quantile Regression
#### Why is the Quantile Regression being used?
The big advantage of a Quantile Regression over a 'normal' OLS Regression is that the Quantile Regression is more robust than the OLS Regression. Furthermore is the assumption of homoscedicity not nessasary for a quantile regression. As the quantile regression does not minimize the ordinary least square loss function such as the OLS regression, but the so called quantile loss, which is less affected by heteroskedasticity. 

In [ ]:
MSE_quant = []
MAE_quant = []
quantiles = [0.25,0.5,0.75]#different quantiles
for i in quantiles:
    mod = smf.quantreg("ST_1 ~ HEA + MAT + RIP + HEI + TGW + EAR + GPE + DRY + HI", df_train) #specify our model
    res = mod.fit(q=i) #fit model
    y_pred = res.predict(X_test) #make predictions
    MSE_quant.append(mean_squared_error(y_test, y_pred)) #calculate different performance meassures
    MAE_quant.append(mean_absolute_error(y_test, y_pred))
    print('Quantile Regression with q=',i,res.summary())

Again we face multicollinearity. This is makes sense as we used the same data and features. We can deal with this in similair fashion as before.

In [ ]:
MSE_quant = []
MAE_quant = []
quantiles = [0.25,0.5,0.75] #different quantiles
for i in quantiles:
    mod = smf.quantreg("ST_1 ~ HEA + MAT + RIP + HEI + EAR + GPE + HI", df_train) #specify our model
    res = mod.fit(q=i) #fit model
    y_pred = res.predict(X_test) #make predictions
    MSE_quant.append(mean_squared_error(y_test, y_pred)) #calculate different performance meassures
    MAE_quant.append(mean_absolute_error(y_test, y_pred))
    print('Quantile Regression with q=',i,res.summary())

## Next part:

Above, we used the $\frac{y_t}{sqrt(y_c)}$ as an index for salt tolerance (as suggested by the paper(s)).
This index is obviously heavily dependent on yield, and is in essence a proxy for salt tolerance, it only considers the yield of the plant.

Outside of food production, it can be beneficial if the plant grows faster, is thicker, produces more cholorophil and so forth. The dataset we have available to us, was curated with the purpose of testing salt tolerance. Despite the author's choice of salt tolerance index, there are still some general plant statistics that are not directly related to yield.

We make some assumptions: primarily, that "more is better". Under these assumptions, is it possible to define a multi-dimensional measure of plant health and/or salt tolerance?

We can model plant health based on the control plants. We can then apply this model to the treatment plants to get a measure for their plant health?

#### Run PCA/Other factor models and interpret the structure
- How many factors do we have?
- What is the factor structure?

The below PCA suggests only a single factor. At most, maybe 2. This is in line with "plant health" and or "salt tolerance" being a relatively simple construct.


In [ ]:
cols = ['HEA', 'MAT', 'RIP', 'HEI', 'TGW', 'EAR', 'GPE', 'DRY']

In [ ]:
dat = dt_t.loc[:,2013,:][cols]/np.sqrt(dt_t.loc[:,2013,:][cols]).values
eigs, vecs = np.linalg.eig(np.cov(dat.T))
plt.plot(range(1,9),sorted(eigs,reverse=True));
plt.xlabel("Dimension");
plt.ylabel("Eigenvalue");
plt.title("Total variance explained");

In [ ]:
pca = PCA()
pca.fit(dt_t.loc[:,2013,:][cols]/np.sqrt(dt_t.loc[:,2013,:][cols]).values)
plt.plot(pca.explained_variance_ratio_);
plt.xlabel("test")

# Recheck the criteria of factor selection. Looks like we have at most 2 main factor.
# Makes sense, as the dataset is supposed to measure Salt Tolerence.
pca.explained_variance_ratio_

In [ ]:
fit_data = dt_t.loc[:,2013,:][cols]/np.sqrt(dt_t.loc[:,2013,:][cols])
fa = FactorAnalyzer(n_factors=1,rotation="varimax")
fa.fit(fit_data)
df_loadings = pd.DataFrame(fa.loadings_, index=cols)
df_loadings


In [ ]:
ST_fa = fit_data @ df_loadings
ST_fa 

### Fit SEM model(s)
- Are there subfactors?
- Is there just one overall factor (in essence linear regression)



##### Notes:

- RIP fucks everything up. Looks like it has no relation to the rest.
- Growth and Yield factors have no real relationship.

Flowering time (HEA), maturity time (MAT), ripening period (RIP), plant height (HEI), thousand grain mass (TGW), ear number per plant (EAR), grain number per ear (GPE), dry mass per m2 (DRY_WT), yield (YLD), and harvest index (HI).

In [ ]:
pd.plotting.scatter_matrix(dt_t.loc[:,2013,:]/np.sqrt(dt_c.loc[:,2013,:]), alpha=0.2, figsize=(20,10));

In [ ]:
dt_c.loc[:,2013,:]

In [ ]:
# RIP is totally unrelated to everything apparently. Why? Idk

desc_reg = \
'''
Salt_Tol =~ EAR + GPE + DRY + TGW + HEA + MAT + HEI + RIP
'''


In [ ]:
mod1 = semopy.Model(desc_reg)
res = mod1.fit(dt_t.loc[:,2013,:]/np.sqrt(dt_c.loc[:,2013,:]), obj="ULS") #Train on 2013 data
print(res)
print(mod1.inspect())

# 1 Loading per factor is set to 1 for identification. 
# The other loadings are there in relative to this loading.

In [ ]:
order = mod1.inspect().iloc[0:8,0].values

In [ ]:
semopy.semplot(mod1, "./model_uni.png",std_ests=True)

rmsea: <0.05
rmr: < 0.05
cfi: >0.95
tli: > 0.97
aic/BIC smaller is better

x2 / df < 2

In [ ]:
semopy.calc_stats(mod1)

In [ ]:
fitdel = mod1.predict_factors(df.loc[:,2014,:]) # Fit to entire dataset

  
ST1 = dt_t.loc[:,2014,:]["YLD"]/np.sqrt(dt_c.loc[:,2014,:]["YLD"]) 
ST = fitdel.G
#ST = (fitst.G/fitsc.ST).values
plt.scatter(ST, ST1);
np.corrcoef(ST, ST1)

In [ ]:

fitdel = mod1.predict_factors(df.loc[:,2013,:]) # Fit to entire dataset

  
ST1 = dt_t.loc[:,2013,:]["YLD"]/np.sqrt(dt_c.loc[:,2013,:]["YLD"]) 
ST = fitdel.G
#ST = (fitst.G/fitsc.ST).values
plt.scatter(ST, ST1);
np.corrcoef(ST, ST1)

In [ ]:
plt.scatter(ST_fa,ST);

In [ ]:
plt.scatter(ST_fa,ST1);

In [ ]:
factors = mod1.inspect().iloc[0:8,[0,3]].set_index("lval")
variances = mod1.inspect().iloc[9:,3].values

In [ ]:
factors.join(df_loadings)

In [ ]:
plt.imshow(np.outer(factors,factors));

In [ ]:
plt.imshow(np.cov(df[order].T));

In [ ]:
# best fit so far
desc1 = \
''' Salt.Tol =~ ST_Dev + ST_Prod
    ST_Prod =~ EAR + GPE + DRY + TGW 
    ST_Dev =~ HEA + MAT + RIP + HEI
'''

In [ ]:
mod2 = semopy.Model(desc1)
res = mod2.fit(dt_t.loc[:,2013,:]/np.sqrt(dt_c.loc[:,2013,:]), obj="ULS", clean_slate=True) #Train on 2013 data
print(res)
print(mod2.inspect())

# 1 Loading per factor is set to 1 for identification. 
# The other loadings are there in relative to this loading.

In [ ]:
semopy.semplot(mod2, "./model_bi.png",plot_covs=True,std_ests=True)

In [ ]:
semopy.calc_stats(mod2)

In [ ]:
fitdel = mod2.predict_factors(df.loc[:,2014,:]) # Fit to entire dataset

  
ST1 = dt_t.loc[:,2014,:]["YLD"]/np.sqrt(dt_c.loc[:,2014,:]["YLD"]) 
ST = fitdel.Yield
#ST = (fitst.G/fitsc.ST).values
plt.scatter(ST, ST1);
np.corrcoef(ST, ST1)

In [ ]:
fitdel = mod2.predict_factors(df.loc[:,2014,:]) # Fit to entire dataset

  
ST1 = dt_t.loc[:,2014,:]["YLD"]/np.sqrt(dt_c.loc[:,2014,:]["YLD"]) 
ST = fitdel.Growth
#ST = (fitst.G/fitsc.ST).values
plt.scatter(ST, ST1);
np.corrcoef(ST, ST1)

In [ ]:
?semopy.semplot

In [ ]:

desc2 = \
''' #ST =~ Growth + Yield 
    ST_Prod =~ EAR + GPE + DRY + TGW + HEI
    ST_Dev =~ HEA + MAT + RIP + HEI + TGW
    ST_Prod ~ ST_Dev

    # Covariances - Something else correlating these, not explained by the model...
    GPE ~~ 0 * GPE
    TGW ~~ HEI
'''

mod3 = semopy.Model(desc2)
res = mod3.fit(dt_t.loc[:,2013,:]/np.sqrt(dt_c.loc[:,2013,:]), obj="ULS") #Train on 2013 data
print(res)
display(mod3.inspect(std_est=True))

# 1 Loading per factor is set to 1 for identification. 
# The other loadings are there in relative to this loading.
display(semopy.semplot(mod3, "./model_sub.png",std_ests=True))
display(semopy.calc_stats(mod3))

In [ ]:
fitdel = mod3.predict_factors(df.loc[:,2014,:]) # Fit to 2014 subset
fitdel
  
#ST1 = dt_t.loc[:,2014,:]["YLD"]/np.sqrt(dt_c.loc[:,2014,:]["YLD"]) 
#ST = fitdel.ST_Yield

#fig, ax = plt.subplots(2,2, figsize=(11,11))
#ax[0,0].scatter(fitdel.ST_Yield, ST1, s=4, alpha=0.5);
#ax[0,1].scatter(fitdel.ST_Growth, ST1, s=4, alpha=0.5);
#ax[1,0].scatter(fitdel.ST_Growth, fitdel.ST_Yield, s=4, alpha=0.5);
#np.corrcoef(ST, ST1)

#### Trying to make the model predict factors... Running into pure linear algebra problems. 
Semopy is a python port of Lavaan, and is not as robustly implemented (yet).

At this point, we move over to R and test the models there.

In [ ]:
lats = mod3.vars['latent']
num_lat = len(lats)
x = df.loc[:,2014,:][mod3.vars['observed']].values.T
lambda_h = mod3.mx_lambda[:,:num_lat]
lambda_x = mod3.mx_lambda[:,num_lat:]
c = np.linalg.inv(np.identity(mod3.mx_beta.shape[0]) - mod3.mx_beta)
c_1 = c[:num_lat, :]
c_2 = c[num_lat:, :]
M_h = x
t = lambda_x @ c_2
L_zh = (t @ mod3.mx_psi @ t.T + mod3.mx_theta) * (x.shape[1])